In [1]:
options(warn=-1)

# import required library
if (!require(lubridate)) {
    install.packages('lubridate')
}
if (!require(forecast)) {
    install.packages('forecast')
}

library(lubridate)  # convert date information
library(forecast)  # make forecast

Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: forecast
also installing the dependencies ‘quadprog’, ‘tseries’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘RcppArmadillo’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


# Data Processing

In [2]:
# load the data
train = read.csv('./train.csv')
test = read.csv('./test.csv')

In [7]:
# transform the date
train$Date = as.Date(train$Date, '%Y-%m-%d')
test$Date = as.Date(test$Date, '%Y-%m-%d')

# get the year and month information
train$year = year(train$Date)
test$year = year(test$Date)

train$month = month(train$Date)
test$month = month(test$Date)

# get week information
train.week = train$Date
train.week = train.week - train.week[1]  # date is now 0, 7, 14, ...
train.week = train.week / 7 + 5  # make 2010-02-05 as '5'
train.week = as.numeric(train.week) %% 52  ## 52 weeks in a year
train$week = train.week

test.week = test$Date
test.week = test.week - test.week[1]
test.week = test.week / 7 + 9 # make 2011-03-04 as '9'.
test.week = as.numeric(test.week) %% 52
test$week = test.week

In [8]:
# output the head of training and test set
head(train)
tail(test)

Store,Dept,Date,Weekly_Sales,IsHoliday,year,month,week
1,1,2010-02-05,24924.50,FALSE,2010,2,5
1,1,2010-02-12,46039.49,TRUE,2010,2,6
1,1,2010-02-19,41595.55,FALSE,2010,2,7
1,1,2010-02-26,19403.54,FALSE,2010,2,8
1,1,2010-03-05,21827.90,FALSE,2010,3,9
1,1,2010-03-12,21043.39,FALSE,2010,3,10


,Store,Dept,Date,Weekly_Sales,IsHoliday,Weekly_Pred1,Weekly_Pred2,Weekly_Pred3,year,month,week
257450,45,98,2012-09-21,467.30,FALSE,0,0,0,2012,9,38
257451,45,98,2012-09-28,508.37,FALSE,0,0,0,2012,9,39
257452,45,98,2012-10-05,628.10,FALSE,0,0,0,2012,10,40
257453,45,98,2012-10-12,1061.02,FALSE,0,0,0,2012,10,41
257454,45,98,2012-10-19,760.01,FALSE,0,0,0,2012,10,42
257455,45,98,2012-10-26,1076.80,FALSE,0,0,0,2012,10,43


# Model 1

* use the information from the previous year
* the same department, same store
* week - 1, week, week + 1

In [5]:
start.time = Sys.time()  # get time information

for (t in 1:2) {
    month = 2 + t
    year = 2011
    if (month > 12) {
        month = month - 12
        year = 2011 + 1
    }
    
    # get the tmp test data
    tmp.test = test[(test$year == year) & (test$month == month), ]
    
    # print useful information
    cat('Current date:\t', year, month, nrow(tmp.test), nrow(train), '\n')
    
    # get the length of unique store and department
    store = sort(unique(tmp.test$Store))
    n.store = length(store)
    dept = sort(unique(tmp.test$Dept))
    n.dept = length(dept)
    
    # choose the median value from the last year, in week - 1, week, and week + 1
    for (s in 1:n.store){
        for (d in 1:n.dept){
            # find the data for (store, dept) = (s, d)
            test.id = which(test$Store == store[s] & test$Dept == dept[d] &
                            test$year == year & test$month == month)
            test.temp = test[test.id, ]
            train.id = which(train$Store == store[s] & train$Dept == dept[d])
            train.temp = train[train.id, ]

            for (i in 1:length(test.id)){
                id = which(train.temp$week == test.temp[i,]$week & 
                           train.temp$year == test.temp[i,]$year - 1)
                # three weeks in the last year
                threeWeeksId = c(id - 1, id, id + 1)  
                tempSales = train.temp[threeWeeksId, 'Weekly_Sales']
                
                if (length(tempSales) == 0){
                    test$Weekly_Pred1[test.id[i]] = 0
                }else{
                    test$Weekly_Pred1[test.id[i]] = median(tempSales)
                }
            }
        }
    }
    
    # read new input file
    tmp.filename = paste('xxx', t, '.csv', sep='');
    newtest = read.csv(tmp.filename)
    
    # merge together
    train = rbind(train[, names(newtest)], newtest)
    
    # transform the date
    train$Date = as.Date(train$Date, '%Y-%m-%d')

    # get the year and month information
    train$year = year(train$Date)
    train$month = month(train$Date)

    # get week information
    train.week = train$Date
    train.week = train.week - train.week[1]  # date is now 0, 7, 14, ...
    train.week = train.week / 7 + 5  # make 2010-02-05 as '5'
    train.week = as.numeric(train.week) %% 52  ## 52 weeks in a year
    train$week = train.week
}

Sys.time() - start.time

Current date:	 2011 3 11813 164115 
Current date:	 2011 4 14746 175928 


Time difference of 4.034444 mins

In [9]:
names(newtest)

[1] "Store"        "Dept"         "Date"         "Weekly_Sales" "IsHoliday"

In [23]:
head(train)

Store,Dept,Date,Weekly_Sales,IsHoliday,year,month,week
1,1,2010-02-05,24924.50,FALSE,2010,2,5
1,1,2010-02-12,46039.49,TRUE,2010,2,6
1,1,2010-02-19,41595.55,FALSE,2010,2,7
1,1,2010-02-26,19403.54,FALSE,2010,2,8
1,1,2010-03-05,21827.90,FALSE,2010,3,9
1,1,2010-03-12,21043.39,FALSE,2010,3,10


In [ ]:
# # get the length of unique store and department
# store = sort(unique(test$Store))
# n.store = length(store)
# dept = sort(unique(test$Dept))
# n.dept = length(dept)

# # choose the median value from the last year, in week - 1, week, and week + 1
# for (s in 1:n.store){
# #     cat("Store: ", store[s], "\n")
#     for (d in 1:n.dept){
#         # find the data for (store, dept) = (s, d)
#         test.id = which(test$Store == store[s] & test$Dept == dept[d])
#         test.temp = test[test.id, ]
#         train.id = which(train$Store == store[s] & train$Dept == dept[d])
#         train.temp = train[train.id, ]
        
#         for (i in 1:length(test.id)){
#             id = which(train.temp$week == test.temp[i,]$week & train.temp$year == test.temp[i,]$year - 1)
#             threeWeeksId = c(id - 1, id, id + 1)  ## three weeks in the last year
#             tempSales = train.temp[threeWeeksId, 'Weekly_Sales']
#             if (length(tempSales) == 0){
#                 test$Weekly_Pred1[test.id[i]] = 0
#             }else{
#                 test$Weekly_Pred1[test.id[i]] = median(tempSales)
#             }
#         }
#     }
# }

# Model 2

* use the information from the previous years
* the same department, but different stores
* use the median of the same week

In [ ]:
# # get the length of unique department
# dept = sort(unique(test$Dept))
# n.dept = length(dept)

In [ ]:
# # choose the median value from the last year, in week - 1, week, and week + 1
# for (d in 1:n.dept){
# #     cat("Department: ", dept[d], "\n")
#     # find the data for (store, dept) = (s, d)
#     test.id = which(test$Dept == dept[d])
#     test.temp = test[test.id, ]
#     train.id = which(train$Dept == dept[d])
#     train.temp = train[train.id, ]

#     for (i in 1:length(test.id)){
#         id = which(train.temp$week == test.temp[i,]$week & train.temp$year < test.temp[i,]$year)
#         tempSales = train.temp[id, 'Weekly_Sales']
#         if (length(tempSales) == 0){
#             test$Weekly_Pred2[test.id[i]] = 0
#         }else{
#             test$Weekly_Pred2[test.id[i]] = median(tempSales)
#         }
#     }
# }

# Model 3

* use the information from the previous years
* the same department, same store
* use all weeks' information to make forecast

# Model 4

* not decided yet

# Performance Testing

* According to the definition of the weighted mean absolute error (WMAE)
* Link to Kaggle: [Walmart Recruiting - Store Sales Forecasting](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting#evaluation)

In [25]:
# define weight w
weight = 4 * test$IsHoliday + 1

# calculate the performance of different models
WMAE1 = sum(weight * abs(test$Weekly_Pred1 - test$Weekly_Sales)) / sum(weight)
WMAE2 = sum(weight * abs(test$Weekly_Pred2 - test$Weekly_Sales)) / sum(weight)
WMAE3 = sum(weight * abs(test$Weekly_Pred3 - test$Weekly_Sales)) / sum(weight)

# output the performance of different models
cat(WMAE1, '\t', WMAE2, '\t', WMAE3, '\n')